In [27]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform
from numba import jit
import numpy as np

In [28]:
iris = pd.read_csv('https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/\
0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv')

In [29]:
for col in iris.columns[:-1]:
    iris[col] = ((iris[col] - iris[col].mean()) / iris[col].std())

In [30]:
distmat = squareform(pdist(iris.to_numpy()[:,:-1]))

In [35]:
@jit
def entropy(d, beta):
    x = - d * beta
    y = np.exp(x)
    ysum = y.sum() - 1.
    if ysum < 1e-50:
        return -1.
    else:
        factor = - 1/(np.log(2.) * ysum)
        return factor * ((y * x) - (y * np.log(ysum))).sum()

In [36]:
ents = []
for i in distmat**2:
    ents.append(entropy(i, 1/(2*5.**2.)))

In [37]:
ents

[7.260206144731453,
 7.260901953507974,
 7.25861433261193,
 7.259009951940271,
 7.259069706348799,
 7.262575868138765,
 7.25820221030839,
 7.26030989380511,
 7.258215983339074,
 7.260217751607517,
 7.261548396264947,
 7.259367448196889,
 7.259664353370557,
 7.255435620284151,
 7.261914390932124,
 7.261671426007698,
 7.261397757636883,
 7.260899880830547,
 7.264151116618894,
 7.259837553426971,
 7.263328257334855,
 7.261043798782499,
 7.254835897801698,
 7.263626632575496,
 7.260350991113473,
 7.261999135324636,
 7.261926236704087,
 7.261155196021406,
 7.261291363872791,
 7.259555342639611,
 7.260513412297405,
 7.263939287962369,
 7.257319774691269,
 7.259362560932771,
 7.260217751607517,
 7.260261146323132,
 7.262539241270895,
 7.260217751607517,
 7.257529602580256,
 7.260942759679762,
 7.259965512606729,
 7.260754473633944,
 7.256680016019234,
 7.262903868627884,
 7.26186446980916,
 7.260917158586571,
 7.259423060387624,
 7.258296657196571,
 7.26089407032346,
 7.260425970173304,
 7.26